<a href="https://colab.research.google.com/github/GeorgeTelles/Backtesting-RSI/blob/main/Backtesting_RSI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Backtesting: Relative Strength Index (RSI)

## Description

This project aims to develop a Python algorithm for backtesting financial assets using the Relative Strength Index (RSI). Backtesting is a crucial technique for evaluating the effectiveness of investment strategies, allowing investors to test their approaches with historical data before applying them in real-time.

## Project Features

- **Data Collection**: Import historical price data of financial assets (stocks, currencies, etc.) from sources such as financial APIs or CSV files.
  
- **RSI Calculation**: Implement the calculation of the Relative Strength Index (RSI) to assess the strength of a price trend and identify overbought or oversold conditions. RSI is calculated based on price changes over a specified period, commonly 14 days.

- **Strategy Simulation**: Test various trading strategies based on RSI, such as buying in oversold conditions and selling in overbought conditions, or crossing critical levels (e.g., 30 and 70).

- **Performance Evaluation**: Measure the performance of strategies using metrics such as total return, drawdown, and other relevant financial metrics.

- **Visualization**: Generate charts to visualize asset prices, RSI values, and buy/sell signals. Visualizations help understand the relationship between price movements and RSI signals.

## Technologies Used

- **Language**: Python

- **Libraries**:
  - `pandas` for data manipulation
  - `numpy` for numerical calculations
  - `matplotlib` and `seaborn` for visualization
  - `requests` or `yfinance` for financial data collection

- **Development Environment**: Jupyter Notebook or your preferred IDE

## Project Structure

1. **Data Collection**: Scripts for importing and cleaning financial data.
   
2. **RSI Calculation**: Implementation of the Relative Strength Index calculation, including the determination of average gains and losses.

3. **Backtesting Engine**: Logic for simulating trading strategies and evaluating performance based on RSI.

4. **Visualization**: Tools for creating charts and detailed reports on RSI performance and applied strategies.

## Disclaimer

This code is provided for demonstration purposes only. The content here is intended to provide supplementary information to assist the investor in making their own investment decisions. None of the topics covered should be construed as any form of endorsement/offer/solicitation for the purchase/sale of any product.

## Documentation for the Library to be Used

[https://vectorbt.dev/](https://vectorbt.dev/)

## What is the Relative Strength Index (RSI)

The Relative Strength Index (RSI) is a widely used technical indicator in financial market analysis to evaluate the speed and change of price movements. Developed by J. Welles Wilder in the 1970s, RSI oscillates between 0 and 100 and is typically calculated over a 14-day period.

- **Overbought and Oversold Conditions**: RSI is used to identify overbought and oversold conditions. Generally, values above 70 indicate overbought conditions, while values below 30 indicate oversold conditions. These conditions can suggest potential reversal points in the market.

- **RSI Formula**: RSI is calculated using the formula:
  
  \[
  RSI = 100 - \frac{100}{1 + RS}
  \]

  Where \( RS \) is the average of gains over the past 14 periods divided by the average of losses over the same periods.

- **Interpretation**: RSI is used to identify potential reversal points in prices and to confirm the strength of a trend. It helps traders recognize moments of potential exhaustion of the current trend.


#1. Importando Bibliotecas e Modulos

In [ ]:
!pip install ta
!pip install vectorbt

In [ ]:
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import numpy as np
import vectorbt as vbt
import ta
import warnings
warnings.filterwarnings('ignore')

# 2. Obtendo dados dos Ativos

In [ ]:
ativo = "ITUB4.SA"

dados_ohlc = yf.download(ativo, "2023-1-1")

#3. Tratando, modelando e separando os dados

In [ ]:
proporcao_treino = 0.70

# Calcular o índice de separação
total_linhas = len(dados_ohlc)
indice_separacao = int(total_linhas * proporcao_treino)

# Dividir o DataFrame
dados_teste = dados_ohlc.iloc[:indice_separacao]
dados_valid = dados_ohlc.iloc[indice_separacao:]

# Verificar os tamanhos dos conjuntos
print(f"Tamanho do conjunto de teste: {len(dados_teste)}")
print(f"Tamanho do conjunto de validação: {len(dados_valid)}")

#4. Definindo metricas a serem testadas

In [ ]:
periodo_rsi = list(range(7, 22, 1))
niveis_entrada = list(range(10, 51, 5))
niveis_saida = list(range(50, 91, 5))


#5. Loop de testes com dados de Teste

In [ ]:
lista_resultados = []
lista_backtest = []

for periodo in periodo_rsi:
  dados_teste2 = dados_teste.copy()
  rsi = ta.momentum.RSIIndicator(dados_teste2['Adj Close'], window = periodo, fillna = False)
  dados_teste2['RSI'] = rsi.rsi()
  dados_teste2 = dados_teste2.dropna()
  for i in niveis_entrada:
        for j in niveis_saida:

          entradas = dados_teste2['RSI'] < i

          saidas = dados_teste2['RSI'] > j

          backtest = vbt.Portfolio.from_signals(dados_teste2['Adj Close'],
                                                    entradas,
                                                    saidas,
                                                    direction='longonly',
                                                    size_type='Amount', size=1)
          retorno_holding = (dados_teste2['Adj Close'].iloc[-1] - dados_teste2['Adj Close'].iloc[0]) / dados_teste2['Adj Close'].iloc[0]

          lista_resultados.append([periodo, i, j, backtest.stats()['Total Return [%]'], backtest.trades.records_readable["Return"].sum()*100, backtest.stats()['Benchmark Return [%]'], retorno_holding*100 ])
          lista_backtest.append(backtest)


# 6. Conferindo os 10 melhores resultados

In [ ]:
resultados = pd.DataFrame(lista_resultados, columns=['Periodo','Entrada', 'Saida', 'Resultado', "Resultado 2", 'Resultado Holding', 'Resultado Holding 2'])
top10 = resultados.sort_values(by='Resultado 2', ascending=False).head(10)
top10

#7. Visualização individual do teste

In [ ]:
lista_backtest[312].plot().show()

In [ ]:
lista_backtest[312].stats()

#8. Teste Apenas com os parametros top10 dos dados de teste

In [ ]:
periodos_top = top10['Periodo'].tolist()
entradas_top = top10['Entrada'].tolist()
saidas_top = top10['Saida'].tolist()

In [ ]:
lista_resultados_teste = []
lista_backtest_teste = []

for i in range(len(entradas_top)):

  dados_valid2 = dados_valid.copy()
  rsi = ta.momentum.RSIIndicator(dados_valid2['Close'], window = periodos_top[i], fillna = False)
  dados_valid2['RSI'] = rsi.rsi()
  dados_valid2 = dados_valid2.dropna()

  entradas = dados_valid2['RSI'] < entradas_top[i]

  saidas = dados_valid2['RSI'] > saidas_top[i]

  backtest = vbt.Portfolio.from_signals(dados_valid2['Close'],
                                            entradas,
                                            saidas,
                                            direction='longonly',
                                            size_type='Amount', size=1)
  lista_resultados_teste.append([periodos_top[i], entradas_top[i], saidas_top[i], backtest.stats()['Total Return [%]'], backtest.trades.records_readable["Return"].sum()*100, backtest.stats()['Benchmark Return [%]'] ])
  lista_backtest.append(backtest)

In [ ]:
resultados_teste = pd.DataFrame(lista_resultados_teste, columns=['Periodo', 'Entrada', 'Saida', 'Resultado','Resultado 2', 'Resultado Holding'])
top10_teste = resultados_teste.sort_values(by='Resultado', ascending=False).head(10)
top10_teste

#9. Backtesting com dados de Validação

In [ ]:
lista_resultados_valid = []
lista_backtest_valid = []

for periodo in periodo_rsi:
  dados_valid2 = dados_valid.copy()
  rsi = ta.momentum.RSIIndicator(dados_valid2['Close'], window = periodo, fillna = False)
  dados_valid2['RSI'] = rsi.rsi()
  dados_valid2 = dados_valid2.dropna()
  for i in niveis_entrada:
        for j in niveis_saida:

          entradas = dados_valid2['RSI'] < i

          saidas = dados_valid2['RSI'] > j

          backtest = vbt.Portfolio.from_signals(dados_valid2['Close'],
                                                    entradas,
                                                    saidas,
                                                    direction='longonly',
                                                    size_type='Amount', size=1)
          if backtest.stats()['Total Return [%]'] > 0:
            lista_resultados_valid.append([periodo, i, j, backtest.stats()['Total Return [%]'], backtest.trades.records_readable["Return"].sum()*100, backtest.stats()['Benchmark Return [%]'] ])
            lista_backtest_valid.append(backtest)


In [ ]:
resultados_valid = pd.DataFrame(lista_resultados_valid, columns=['Periodo','Entrada', 'Saida', 'Resultado','Resultado 2', 'Resultado Holding'])
top10_valid = resultados_valid.sort_values(by='Resultado', ascending=False).head(10)
top10_valid

In [ ]:
dados_valid2

**Verificando se existem parametros iguais entre os top 10 dos dados de teste e o top 10 dos dados de validação**

In [ ]:
common_rows = pd.merge(top10_valid, top10, on=['Periodo', 'Entrada', 'Saida'])

# Imprimir as linhas comuns
print("Linhas que são iguais em ambos os dataframes:")
common_rows